<a href="https://colab.research.google.com/github/zyang63/Die_casting_ejection/blob/main/ejection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%%capture
!pip install trimesh
!pip install bpy
!pip install numpy pillow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
blender is already the newest version (3.0.1+dfsg-7).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [76]:
import os
import bpy
import math
import numpy as np
from PIL import Image, ImageDraw
import cv2

In [18]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
#@title #File Entry { display-mode: "form"}
#@markdown User can choose to upload the file to colab directly or select the google file upload button at the bottom of this form. Also choose the number of elements.

button_pressed = False  # Initialize the variable as False
filename = ""
button = widgets.Button(description="Google upload dialog")
output = widgets.Output()

def on_button_clicked(b):
    global button_pressed  # Access the global variable
    global filename
    with output:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        button_pressed = True  # Set the variable to True when the button is clicked

button.on_click(on_button_clicked)
display(button, output)

Button(description='Google upload dialog', style=ButtonStyle())

Output()

In [77]:
#@title #Element Count { display-mode: "form", run: "auto" }
element_count = 500 #@param {type:"slider", min:10, max:500, step:1}
if not button_pressed:
  filename = "/content/Balance_Shaft_Housing_Full_Shot parts - 8M0111257.stl" #@param {type:"string"}
geometry = t.load_mesh(filename)
#geometry = t.load_mesh("/content/ball.stl")
voxel_size = geometry.extents.max()/element_count
print("Element size is in units from stl file ",voxel_size, " per cell")

Element size is in units from stl file  0.35415894317626956  per cell


In [78]:
rotation_angle = 90
Mu = 0.6
unit = 0.001

In [79]:
obj_file_path = '/content/remeshed_uv.obj'
blender_file_path = '/content/meshed_geometry.blend'
ejection_coefficient_map_path = '/content/ejection_coefficient_map.png'
parting_line_map_path = '/content/parting_line_map.png'
ejection_coefficient_result_path = '/content/colored_ejection_coefficient.glb'
parting_line_result_path = '/content/colored_parting_line.glb'

# Remesh and output OBJ file

In [80]:
bpy.ops.wm.read_factory_settings(use_empty=True)
bpy.ops.import_mesh.stl(filepath= filename, global_scale = unit)
bpy.ops.object.modifier_add(type='REMESH')
bpy.context.object.modifiers["Remesh"].mode = 'VOXEL'
bpy.context.object.modifiers["Remesh"].voxel_size = voxel_size * unit
bpy.ops.object.modifier_apply(modifier="Remesh")
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.uv.smart_project()
bpy.ops.object.mode_set(mode='OBJECT')
selected_object = bpy.context.object
for obj in bpy.context.selected_objects:
    if obj.type == "MESH":
        bpy.ops.wm.obj_export(filepath= obj_file_path, export_triangulated_mesh=False, export_materials=False)
bpy.ops.wm.save_as_mainfile(filepath = blender_file_path)

Import finished in 12.9498 sec.
Info: Saved "meshed_geometry.blend"


{'FINISHED'}

# Process OBJ file

In [81]:
def process_obj(obj_file_path):
    vertices, faces, vt_coordinates, vt_faces = [], [], [], []
    with open(obj_file_path, 'r') as obj_file:
        for line in obj_file:
            parts = line.split()
            if parts[0] == 'v':
                vertices.append(list(map(float, parts[1:4])))
            elif parts[0] == 'f':
                f, vt_f = zip(*((int(x.split('/')[0]) - 1, int(x.split('/')[1]) - 1) for x in parts[1:]))
                faces.append(f), vt_faces.append(vt_f)
            elif parts[0] == 'vt':
                vt_coordinates.append(list(map(float, parts[1:])))
    return vertices, faces, vt_coordinates, vt_faces

In [82]:
vertices, faces, vt_coordinates, vt_faces = process_obj(obj_file_path)

# Ejection coefficient calculation

In [83]:
def ejection_coefficient_calculation(selected_axis_data, vertices, faces, Mu, unit):
    direction, num_eject_faces = np.array(selected_axis_data), 0
    a_list, Phi_list = [], []
    for face in faces:
        normal = np.cross(*(np.array(vertices[face[i]]) - np.array(vertices[face[0]]) for i in (1, 2)))
        cosine = np.dot(normal / np.linalg.norm(normal), direction / np.linalg.norm(direction))
        a = Mu * np.sqrt(1 - cosine**2) - cosine
        num_eject_faces += (0 <= a <= Mu)
        a_list.append(a)
    Phi_list = [num_eject_faces * (unit**2) * a for a in a_list]
    return Phi_list, max((Phi for Phi in Phi_list if Phi <= Mu * num_eject_faces * (unit**2)), default=None)

#Optimize ejection direction


In [84]:
def ejection_force_rotation(rotation_angle):
    def rotation_matrix(axis, theta):
        theta = np.radians(theta)
        if axis == 'x':
            return np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
        elif axis == 'y':
            return np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
        elif axis == 'z':
            return np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    unique_rotations = {}
    for theta_x in range(0, 360 + rotation_angle, rotation_angle):
        for theta_y in range(0, 360 + rotation_angle, rotation_angle):
            for theta_z in range(0, 360 + rotation_angle, rotation_angle):
                rotated_vector = np.dot(rotation_matrix('x', theta_x),
                                        np.dot(rotation_matrix('y', theta_y),
                                               np.dot(rotation_matrix('z', theta_z), np.array([0, 1, 0]))))
                rotated_vector = tuple(np.where(np.abs(rotated_vector) < 0.001, 0, rotated_vector))
                unique_rotations.setdefault(rotated_vector, (theta_x, theta_y, theta_z))
    unique_angles_list = list(unique_rotations.values())
    unique_vectors_list = [list(vec) for vec in unique_rotations.keys()]
    return unique_angles_list, unique_vectors_list

In [ ]:
directions = ejection_force_rotation(rotation_angle)[1]
rotation_max_Phi = [ejection_coefficient_calculation(direction, vertices, faces, Mu, unit)[-1] for direction in directions]
min_Phi, ejection_direction = min(zip(rotation_max_Phi, directions))
Phi_list_optimize, max_Phi_optimize = ejection_coefficient_calculation(ejection_direction, vertices, faces, Mu, unit)

<ipython-input-83-b03a15596a11>:6: RuntimeWarning: invalid value encountered in divide
  cosine = np.dot(normal / np.linalg.norm(normal), direction / np.linalg.norm(direction))


In [ ]:
print(ejection_direction)

In [ ]:
unique_angles_list, unique_vectors_list = ejection_force_rotation(rotation_angle)
print(unique_angles_list)
print(unique_vectors_list)
print(rotation_max_Phi)
print(ejection_direction)
print(max_Phi_optimize)

# Core?

In [ ]:
# Required Libraries
import numpy as np

# Load the .obj file
def load_obj(file_path):
    vertices = []
    faces = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertex = list(map(float, parts[1:4]))
                vertices.append(vertex)
            elif line.startswith('f '):
                parts = line.split()
                # Adjusting for faces using 1-indexed OBJ format
                face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                faces.append(face)
    return np.array(vertices), faces

# Check if a face has a value greater than 0.6
def filter_faces_by_value(faces, values, threshold=max_Phi_list):
    filtered_faces = [face for face, value in zip(faces, values) if value > threshold]
    return filtered_faces

# Find vertices in a specific direction from given faces
def find_vertices_in_direction(vertices, faces, direction):
    direction = np.array(direction)
    selected_vertices = []
    for face in faces:
        for vertex_idx in face:
            vertex = vertices[vertex_idx]
            if np.dot(vertex, direction) > 0:  # Change this condition as needed
                selected_vertices.append(vertex_idx)
    return set(selected_vertices)

# Check if faces formed by vertices have values smaller than 0.6
def check_face_values(vertices, faces, values, selected_vertices, threshold=max_Phi_list):
    for face, value in zip(faces, values):
        face_vertices = set(face)
        if face_vertices.issubset(selected_vertices):
            if value < threshold:
                return "have cores"
    return "no cores"

vertices, faces = load_obj('/content/remeshed_uv.obj')

# Replace with your actual face values (must be in the same order as `faces`)
face_values = np.array(Phi_list)

# Step 1: Find faces larger than 0.6
faces_larger_than_06 = filter_faces_by_value(faces, face_values, threshold= max_Phi_list)

# Step 2: Find vertices in a specific direction for these faces
direction = ejection_direction
selected_vertices = find_vertices_in_direction(vertices, faces_larger_than_06, direction)

# Step 3 & 4: Check if any face value is smaller than 0.6
result = check_face_values(vertices, faces, face_values, selected_vertices, threshold=max_Phi_list)

# Output result
print(result)


no cores


In [ ]:
# Load or create your mesh
mesh = t.load('/content/remeshed_uv.obj')

# Get the bounds
min_bound, max_bound = mesh.bounds

print("Maximum bound:", max_bound)
print("Minimum bound:", min_bound)

Maximum bound: [8.76565  0.100003 4.882059]
Minimum bound: [ -8.76565   -3.66875  -12.822659]


In [ ]:
import numpy as np

# Load the obj file data
def load_obj(file_path):
    vertices = []
    faces = []
    face_values = np.array(Phi_list)

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):  # Vertex line
                vertices.append(list(map(float, line.split()[1:4])))
            elif line.startswith('f '):  # Face line
                indices = [int(idx.split('/')[0]) - 1 for idx in line.split()[1:5]]
                faces.append(indices)

    vertices = np.array(vertices)
    faces = np.array(faces)
    face_values = np.array(face_values)
    return vertices, faces, face_values

# Load vertices, faces, and face values
file_path = '/content/remeshed_uv.obj'  # Replace with your OBJ file path
vertices, faces, face_values = load_obj(file_path)

# Parameters
#max_Phi_list = 0.5  # Replace with your threshold
direction = np.array([0.0, -1.0, 0.0])

# Step 1: Find faces larger than max_Phi_list
faces_above_threshold = faces[face_values > max_Phi_list]
values_above_threshold = face_values[face_values > max_Phi_list]

# Step 2: Move faces along the direction until all vertices exceed max_bound
def move_face_out_of_boundary(face, vertices, direction, max_bound, min_bound):
    face_vertices = vertices[face]

    # Calculate the scaling factors for each vertex in each dimension
    scale_factors = []
    for vertex in face_vertices:
        # Calculate the scaling factors needed to reach max_bound and min_bound for each dimension
        scale_to_max = np.where(direction > 0, (max_bound - vertex) / direction, np.inf)
        scale_to_min = np.where(direction < 0, (min_bound - vertex) / direction, np.inf)

        # Get the minimum positive scaling factor for this vertex to move out of bounds
        scale = np.min(np.where(direction != 0, np.minimum(scale_to_max, scale_to_min), np.inf))
        scale_factors.append(scale)

    # Use the maximum of these minimum scales to ensure the face moves fully out of the boundary
    max_scale = max(scale_factors)

    # Move the face out of the boundary
    moved_face = face_vertices + max_scale * direction
    return moved_face

In [ ]:
moved_faces = [move_face_out_of_boundary(face, vertices, direction, max_bound, min_bound) for face in faces_above_threshold]
count = len(moved_faces)
print("Total elements in the list:", count)

<ipython-input-222-dd9034fc908b>:42: RuntimeWarning: divide by zero encountered in divide
  scale_to_max = np.where(direction > 0, (max_bound - vertex) / direction, np.inf)
<ipython-input-222-dd9034fc908b>:43: RuntimeWarning: divide by zero encountered in divide
  scale_to_min = np.where(direction < 0, (min_bound - vertex) / direction, np.inf)
<ipython-input-222-dd9034fc908b>:43: RuntimeWarning: invalid value encountered in divide
  scale_to_min = np.where(direction < 0, (min_bound - vertex) / direction, np.inf)
<ipython-input-222-dd9034fc908b>:42: RuntimeWarning: invalid value encountered in divide
  scale_to_max = np.where(direction > 0, (max_bound - vertex) / direction, np.inf)


Total elements in the list: 120025


In [ ]:
first_element = moved_faces[0]
print("First element:", first_element)

First element: [[-8.473079 -3.668961  1.07498 ]
 [-8.448504 -3.733717  1.081469]
 [-8.432911 -3.711685  1.114669]
 [-8.43784  -3.66875   1.119549]]


In [ ]:
from itertools import combinations

def check_faces_in_region(start_face_indices, moved_face_indices, vertices, faces, direction):
    # Step 1: Define the axis-aligned bounding box (AABB) between start_face and moved_face
    start_face_vertices = vertices[start_face_indices]
    moved_face_vertices = moved_face_indices

    bounding_box_min = np.minimum(np.min(start_face_vertices, axis=0), np.min(moved_face_vertices, axis=0))
    bounding_box_max = np.maximum(np.max(start_face_vertices, axis=0), np.max(moved_face_vertices, axis=0))

    # Filter vertices within the AABB
    vertices_in_bbox_mask = np.all((vertices >= bounding_box_min) & (vertices <= bounding_box_max), axis=1)
    vertices_in_bbox = vertices[vertices_in_bbox_mask]

    # Step 2: Project start_face and moved_face vertices onto the direction vector
    start_projection = np.dot(start_face_vertices, direction)
    moved_projection = np.dot(moved_face_vertices, direction)
    min_projection, max_projection = min(start_projection.min(), moved_projection.min()), max(start_projection.max(), moved_projection.max())

    # Further filter vertices within the AABB to keep those between start and moved projections along direction
    valid_vertices_mask = (np.dot(vertices_in_bbox, direction) >= min_projection) & (np.dot(vertices_in_bbox, direction) <= max_projection)
    valid_vertices = vertices_in_bbox[valid_vertices_mask]

    # Step 3: Find the order of valid vertices in the original vertices array
    valid_vertices_order = [np.where((vertices == v).all(axis=1))[0][0] for v in valid_vertices]
    print(valid_vertices_order)

    # Generate a set of all three-element combinations of valid vertices indices for quick lookup
    valid_combinations = {frozenset(combo) for combo in combinations(valid_vertices_order, 4)}

    # Step 4: Filter faces by face_values and check combinations
    for i, face in enumerate(faces):
        if face_values[i] >= max_Phi_list:  # Skip faces that do not meet the value criteria
            continue

        face_set = set(face)
        # Check if any three-element subset of this face matches a valid combination
        for combo in combinations(face_set, 4):
            if frozenset(combo) in valid_combinations:
                print("Has core in faces:", i)
                return True  # Exit as soon as we find a match

    # Output result if no core is found
    print("No core found")
    return False


# Check for faces within each region
found_cores = False
for start_face_indices, moved_face_indices in zip(faces_above_threshold, moved_faces):
    if check_faces_in_region(start_face_indices, moved_face_indices, vertices, faces, direction):
        found_cores = True
        break  # Exit the loop if a core is found


[4, 5, 10, 11]
No core found
[5, 6, 10, 11, 12]
No core found
[1, 2, 11, 12]
No core found
[14, 15, 21, 22]
No core found
[4, 5, 21, 22]
No core found
[15, 16, 22, 23]
No core found
[5, 6, 22, 23]
No core found
[18, 19, 26, 27]
No core found
[19, 20, 26, 27, 28]
No core found
[8, 9, 27, 28]
No core found
[20, 21, 28, 29]
No core found
[9, 10, 28, 29]
No core found
[4, 10, 21, 28, 29]
No core found
[95, 96, 97, 98]
No core found
[97, 98, 99, 100]
No core found
[92, 93, 99, 100]
No core found
[91, 93, 98, 100]
No core found
[102, 103, 104, 105]
No core found
[104, 105, 106, 107]
No core found
[95, 96, 106, 107]
No core found
[94, 96, 105, 107]
No core found
[95, 97, 106, 108]
No core found
[97, 99, 108, 109]
No core found
[117, 118, 125, 126]
No core found
[118, 119, 125, 126, 127]
No core found
[119, 120, 125, 126, 127, 128]
No core found
[120, 121, 125, 126, 127, 128, 129]
No core found
[121, 122, 125, 126, 127, 128, 129, 130]
No core found
[122, 123, 125, 126, 127, 128, 129, 130, 131]

KeyboardInterrupt: 

In [ ]:
import numpy as np
from itertools import combinations

def check_faces_in_region(start_face_indices, moved_face_indices, vertices, faces, direction, face_values, max_Phi_list):
    # Step 1: Define the AABB for the region
    start_face_vertices = vertices[start_face_indices]
    moved_face_vertices = moved_face_indices

    bounding_box_min = np.minimum(np.min(start_face_vertices, axis=0), np.min(moved_face_vertices, axis=0))
    bounding_box_max = np.maximum(np.max(start_face_vertices, axis=0), np.max(moved_face_vertices, axis=0))

    # Filter vertices within the bounding box
    vertices_in_bbox_mask = np.all((vertices >= bounding_box_min) & (vertices <= bounding_box_max), axis=1)
    vertices_in_bbox = vertices[vertices_in_bbox_mask]

    # Step 2: Project the vertices onto the direction vector and filter
    projections = np.dot(vertices_in_bbox, direction)
    start_projection, moved_projection = np.dot(start_face_vertices, direction), np.dot(moved_face_vertices, direction)
    min_projection, max_projection = min(start_projection.min(), moved_projection.min()), max(start_projection.max(), moved_projection.max())

    valid_vertices_mask = (projections >= min_projection) & (projections <= max_projection)
    valid_vertices = vertices_in_bbox[valid_vertices_mask]

    # Step 3: Find indices of valid vertices in the original array
    valid_vertices_indices = np.where(vertices_in_bbox_mask)[0][valid_vertices_mask]

    # Generate valid 4-combinations
    valid_combinations = {frozenset(combo) for combo in combinations(valid_vertices_indices, 4)}

    # Step 4: Check faces against valid combinations
    for i, face in enumerate(faces):
        if face_values[i] >= max_Phi_list:
            continue

        face_set = frozenset(face)
        # Only check if the face has any subset in the valid_combinations
        if any(frozenset(combo) in valid_combinations for combo in combinations(face_set, 4)):
            print("Has core in faces:", i)
            return True

    print("No core found")
    return False


# Main check loop
found_cores = False
for start_face_indices, moved_face_indices in zip(faces_above_threshold, moved_faces):
    if check_faces_in_region(start_face_indices, moved_face_indices, vertices, faces, direction, face_values, max_Phi_list):
        found_cores = True
        break  # Exit the loop if a core is found

Streaming output truncated to the last 5000 lines.
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core found
No core fou

In [ ]:
# Required Libraries
import numpy as np

# Load the .obj file
def load_obj(file_path):
    vertices = []
    faces = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertex = list(map(float, parts[1:4]))
                vertices.append(vertex)
            elif line.startswith('f '):
                parts = line.split()
                # Adjusting for faces using 1-indexed OBJ format
                face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                faces.append(face)
    return np.array(vertices), faces

# Check if a face has a value greater than 0.6
def filter_faces_by_value(faces, values, threshold = max_Phi_list):
    filtered_faces = [face for face, value in zip(faces, values) if value > threshold]
    return filtered_faces

# Find vertices in a specific direction from given faces
def find_vertices_in_direction(vertices, faces, direction):
    direction = np.array(direction)
    selected_vertices = []
    for face in faces:
        for vertex_idx in face:
            vertex = vertices[vertex_idx]
            if np.dot(vertex, direction) > 0:  # Change this condition as needed
                selected_vertices.append(vertex_idx)
    return set(selected_vertices)

# Check if faces formed by vertices have values smaller than 0.6
def check_face_values(vertices, faces, values, selected_vertices, threshold = max_Phi_list):
    for face, value in zip(faces, values):
        face_vertices = set(face)
        if face_vertices.issubset(selected_vertices):
            if value < threshold:
                return "have cores"
    return "no cores"

# Load your .obj file (update the file path)
vertices, faces = load_obj('/content/remeshed_uv.obj')

# Replace with your actual face values (must be in the same order as `faces`)
face_values = np.array(Phi_list)

# Step 1: Find faces larger than 0.6
faces_larger_than_06 = filter_faces_by_value(faces, face_values, max_Phi_list)

# Step 2: Find vertices in a specific direction for these faces
directions = [-x for x in ejection_direction],ejection_direction

results = []

for direction in directions:
    selected_vertices = find_vertices_in_direction(vertices, faces_larger_than_06, direction)
    result = check_face_values(vertices, faces, face_values, selected_vertices, max_Phi_list)
    results.append(result)
print(results)
# Step 3: Output result based on direction checks
if "no cores" in results:
    print("no cores")
else:
    print("have cores")


['have cores', 'no cores']
no cores


In [ ]:
# Required Libraries
import numpy as np

# Load the .obj file
def load_obj(file_path):
    vertices = []
    faces = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.split()
                vertex = list(map(float, parts[1:4]))
                vertices.append(vertex)
            elif line.startswith('f '):
                parts = line.split()
                # Adjusting for faces using 1-indexed OBJ format
                face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                faces.append(face)
    return np.array(vertices), faces

# Check if a face has a value greater than 0.6
def filter_faces_by_value(faces, values, threshold = max_Phi_list):
    filtered_faces = [face for face, value in zip(faces, values) if value < threshold]
    return filtered_faces

# Find vertices in a specific direction from given faces
def find_vertices_in_direction(vertices, faces, direction):
    direction = np.array(direction)
    selected_vertices = []
    for face in faces:
        for vertex_idx in face:
            vertex = vertices[vertex_idx]
            if np.dot(vertex, direction) < 0:  # Change this condition as needed
                selected_vertices.append(vertex_idx)
    return set(selected_vertices)

# Check if faces formed by vertices have values smaller than 0.6
def check_face_values(vertices, faces, values, selected_vertices, threshold = max_Phi_list):
    for face, value in zip(faces, values):
        face_vertices = set(face)
        if face_vertices.issubset(selected_vertices):
            if value > threshold:
                return "have cores"
    return "no cores"

# Load your .obj file (update the file path)
vertices, faces = load_obj('/content/remeshed_uv.obj')

# Replace with your actual face values (must be in the same order as `faces`)
face_values = np.array(Phi_list)

# Step 1: Find faces larger than 0.6
faces_larger_than_06 = filter_faces_by_value(faces, face_values, max_Phi_list)

# Step 2: Find vertices in a specific direction for these faces
directions = [-x for x in ejection_direction],ejection_direction

results = []

for direction in directions:
    selected_vertices = find_vertices_in_direction(vertices, faces_larger_than_06, direction)
    result = check_face_values(vertices, faces, face_values, selected_vertices, max_Phi_list)
    results.append(result)
print(results)
# Step 3: Output result based on direction checks
if "no cores" in results:
    print("no cores")
else:
    print("have cores")

['have cores', 'have cores']
have cores


# Ejection coefficient map

In [ ]:
image_size = 4000
color_bins = [(i, 0, 256 - i) for i in range(0, 256, 26)]
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates]
for vt_face, Phi in zip(vt_faces, Phi_list_optimize):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        if 0 <= Phi <= max_Phi_optimize:
            color = color_bins[min(int((Phi / max_Phi_optimize) * 10), 9)]
        elif Phi < 0:
            color = (0, 0, 0)
        else:
            color = (120, 120, 120)
        draw.polygon(vt_indices, outline=color, fill=color)
image.save(ejection_coefficient_map_path)

# parting line identified

In [ ]:
image_size = 4000
color_bins = [(i, 0, 256 - i) for i in range(0, 256, 26)]
image = Image.new("RGB", (image_size, image_size), "black")
draw = ImageDraw.Draw(image)
scaled_vt_coordinates = [
    (vt[0] * image_size, (1 - vt[1]) * image_size) for vt in vt_coordinates]
for vt_face, Phi in zip(vt_faces, Phi_list_optimize):
    if all(0 <= j < len(scaled_vt_coordinates) for j in vt_face):
        vt_indices = [scaled_vt_coordinates[j] for j in vt_face]
        color = (256, 256, 256) if Phi > max_Phi_optimize else (256, 0, 0)
        draw.polygon(vt_indices, outline=color, fill=color)
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
red_mask = cv2.inRange(image, np.array([0, 0, 255]), np.array([0, 0, 255]))
white_mask = cv2.inRange(image, np.array([255, 255, 255]), np.array([255, 255, 255]))
kernel = np.ones((3, 3), np.uint8)
red_mask_cleaned_close = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel,iterations=1)
red_mask_cleaned_open = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel,iterations=1)
white_mask_cleaned_close = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel,iterations=1)
white_mask_cleaned_open = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel,iterations=1)
white_dilated = cv2.dilate(white_mask_cleaned_close + white_mask_cleaned_open - white_mask, kernel, iterations=1)
boundary_mask = cv2.bitwise_and(red_mask_cleaned_close + red_mask_cleaned_open - red_mask, white_dilated)
boundary_mask_cleaned_open = cv2.morphologyEx(boundary_mask, cv2.MORPH_OPEN, kernel,iterations=1)
boundary_mask_cleaned_close = cv2.morphologyEx(boundary_mask, cv2.MORPH_CLOSE, kernel,iterations=1)
output_image = np.zeros_like(image)
contours, _ = cv2.findContours(boundary_mask_cleaned_close - boundary_mask_cleaned_open, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(output_image, contours, -1, [0, 255, 0], thickness=2)
cv2.imwrite(parting_line_map_path, cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))

# Visualization

## Ejection coefficient result

In [ ]:
bpy.ops.wm.open_mainfile(filepath = blender_file_path)
material = bpy.data.materials.new(name="MyMaterial")
material.use_nodes = True
nodes = material.node_tree.nodes
links = material.node_tree.links
nodes.clear()
bsdf = nodes.new('ShaderNodeBsdfPrincipled')
texture = nodes.new('ShaderNodeTexImage')
output = nodes.new('ShaderNodeOutputMaterial')
texture.location = (-200, 0)
output.location = (200, 0)
texture.image = bpy.data.images.load(ejection_coefficient_map_path)
links.new(texture.outputs['Color'], bsdf.inputs['Base Color'])
links.new(bsdf.outputs['BSDF'], output.inputs['Surface'])
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath = ejection_coefficient_result_path)

## parting line result

In [ ]:
bpy.ops.wm.open_mainfile(filepath = blender_file_path)
material = bpy.data.materials.new(name="MyMaterial")
material.use_nodes = True
nodes = material.node_tree.nodes
links = material.node_tree.links
nodes.clear()
bsdf = nodes.new('ShaderNodeBsdfPrincipled')
texture = nodes.new('ShaderNodeTexImage')
output = nodes.new('ShaderNodeOutputMaterial')
texture.location = (-200, 0)
output.location = (200, 0)
texture.image = bpy.data.images.load(parting_line_map_path)
links.new(texture.outputs['Color'], bsdf.inputs['Base Color'])
links.new(bsdf.outputs['BSDF'], output.inputs['Surface'])
bpy.context.object.data.materials.append(material)
bpy.ops.export_scene.gltf(filepath = parting_line_result_path)

# Clean

In [ ]:
files_to_delete = [
    "/content/meshed_geometry.blend",
    "/content/remeshed_uv.obj",]
for file_path in files_to_delete:
    if os.path.exists(file_path):
        os.remove(file_path)